## Black Box Approach for all 10 genres classification

We aim at training a Neural Network to distinguish between all the different genres.
In this model we try and augment the dataset

In [1]:
#importing google drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
#setting the working directory
%cd /gdrive/MyDrive/polimi/NAML/NAML_proj/

/gdrive/MyDrive/polimi/NAML/NAML_proj


In [15]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import random
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
import librosa
import os
import copy

tfk = tf.keras
tfkl = tf.keras.layers
seed = 42

The dataset is composed of Mel-Spectrograms of each audio sample, labeled with respect to each genre

In [4]:
dataset = []
genres = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 
        'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}
n_genres = 10

for genre, genre_number in genres.items():
    for filename in os.listdir(f'dataset_old/genres/{genre}'):
        songname = f'dataset_old/genres/{genre}/{filename}'
        for i in range(3):
          y, sr = librosa.load(songname, mono=True, duration=9.9, offset = i*9.9)
          ps = librosa.feature.melspectrogram(y=y, sr=sr, hop_length = 256, n_fft = 512)
          ps = librosa.power_to_db(ps**2)
          dataset.append( (ps, genre_number) )
    print(str(genre+' done'))

blues done
classical done
country done
disco done
hiphop done
jazz done
metal done
pop done
reggae done
rock done


We split the dataset according to the following composition:


*   70% training set
*   20% validation set
*   10% test set

Maintaining equal proportions amongst classes



In [10]:
order = np.arange(start = 0, stop = 100, step = 1)

np.random.seed(seed)

training_validation = []
test = []

for i in range(n_genres):
  shuffle = np.random.permutation(order)
  for k in range(90):
    for kk in range(3):
      training_validation.append(dataset[i*100 + shuffle[k]*3 + kk])
  for l in range(10):
    for ll in range(3):
      test.append(dataset[i*100 + shuffle[l+90]*3 + ll])

In [11]:
X_train_val, Y_train_val = zip(*training_validation)
X_test, Y_test = zip(*test)

X_train_val = np.array([x.reshape( (128, 853, 1) ) for x in X_train_val])
X_test = np.array([x.reshape( (128, 853, 1) ) for x in X_test])

Y_train_val = np.array(tfk.utils.to_categorical(Y_train_val, n_genres))
Y_test = np.array(tfk.utils.to_categorical(Y_test, n_genres))

The proposed model is composed as a stack of convolutional layer, followed by a Global Average Pooling layer leading to a fully connected section.

Optimization is performed as a Batch version of Adam optimizer algorithm

In [12]:
def build_model(input_shape, n_units):
# Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    conv1 = tfkl.Conv2D(
        name = 'conv_1',
        filters=4,
        kernel_size=(5, 5),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(input_layer)
    conv1_2 = tfkl.Conv2D(
        name = 'conv_1_2',
        filters=8,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(conv1)
    pool1 = tfkl.MaxPooling2D(
        name = 'pool_1',
        pool_size = (2, 2)
    )(conv1_2)

    conv2 = tfkl.Conv2D(
        name = 'conv_2',
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool1)
    conv2_2 = tfkl.Conv2D(
        name = 'conv_2_2',
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(conv2)
    pool2 = tfkl.MaxPooling2D(
        name = 'pool_2',
        pool_size = (2, 2)
    )(conv2_2)

    conv3 = tfkl.Conv2D(
        name = 'conv_3',
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool2)
    pool3 = tfkl.MaxPooling2D(
        name = 'pool_3',
        pool_size = (2, 2)
    )(conv3)

    conv4 = tfkl.Conv2D(
        name = 'conv_4',
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool3)
    pool4 = tfkl.MaxPooling2D(
        name = 'pool_4',
        pool_size = (2, 2)
    )(conv4)

    conv5 = tfkl.Conv2D(
        name = 'conv_5',
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool4)
    pool5 = tfkl.MaxPooling2D(
        name = 'pool_5',
        pool_size = (2, 2)
    )(conv5)

    conv6 = tfkl.Conv2D(
        name = 'conv_6',
        filters=286,
        kernel_size=(1, 1),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool5)
    
    global_average = tfkl.GlobalAveragePooling2D(name = 'GAP')(conv6)
    global_average = tfkl.Dropout(0.3, seed=seed, name = 'Dropout_GAP')(global_average)
    
    classifier_layer = tfkl.Dense(units=64, name='Classifier', activation='relu')(global_average)
    
    classifier_layer = tfkl.Dropout(0.3, seed=seed, name = 'Dropout_Classifier')(classifier_layer)
    classifier_layer_2 = tfkl.Dense(units=32, name='Classifier_2', activation='relu')(classifier_layer)
    classifier_layer_2 = tfkl.Dropout(0.3, seed=seed, name = 'Dropout_Classifier_2')(classifier_layer_2)
    output_layer = tfkl.Dense(units=n_units, activation='softmax', name='Output')(classifier_layer_2)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model

    return model

In [13]:
input_shape = (128, 853, 1)

model = build_model(input_shape, n_genres)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 128, 853, 1)]     0         
                                                                 
 conv_1 (Conv2D)             (None, 128, 853, 4)       104       
                                                                 
 conv_1_2 (Conv2D)           (None, 128, 853, 8)       296       
                                                                 
 pool_1 (MaxPooling2D)       (None, 64, 426, 8)        0         
                                                                 
 conv_2 (Conv2D)             (None, 64, 426, 16)       1168      
                                                                 
 conv_2_2 (Conv2D)           (None, 64, 426, 16)       2320      
                                                                 
 pool_2 (MaxPooling2D)       (None, 32, 213, 16)       0     

In [17]:
metrics_accuracy = []
metrics_precision = []
metrics_recall = []
metrics_f1 = []

n_trials = 10
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
adaptive_LR = tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-4)


order = np.arange(start = 0, stop = 90, step = 1)
np.random.seed(seed)
#initialization
X_train = copy.deepcopy(X_train_val[0:(3*700)])
Y_train = copy.deepcopy(Y_train_val[0:(3*700)])
X_valid = copy.deepcopy(X_train_val[0:(3*200)])
Y_valid = copy.deepcopy(Y_train_val[0:(3*200)])


for i in range(n_trials):
  model = build_model(input_shape, n_genres)
  
  #random split in training and validation
  
  for j in range(n_genres):
    shuffle = np.random.permutation(order)
    for k in range(70):
      for kk in range(3):
        X_train[k] = copy.deepcopy(X_train_val[i*90 + shuffle[k]*3 + kk])
        Y_train[k] = copy.deepcopy(Y_train_val[i*90 + shuffle[k]*3 + kk])
    for m in range(20):
      for mm in range(3):
        X_valid[m] = copy.deepcopy(X_train_val[i*90 + shuffle[m+70]*3 + mm])
        Y_valid[m] = copy.deepcopy(Y_train_val[i*90 + shuffle[m+70]*3 + mm])

  standard_history = model.fit(
    x = X_train,
    y = Y_train,
    epochs = 500,
    batch_size = 100,
    validation_data= (X_valid, Y_valid),
    callbacks = [early_stopping, adaptive_LR]
    )
  predictions = model.predict(X_test)
  metrics_accuracy.append(accuracy_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1)))
  metrics_precision.append(precision_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro'))
  metrics_recall.append(recall_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro'))
  metrics_f1.append(f1_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro'))
  model.save(str('/gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_2_'+str(i)))
  print(metrics_accuracy)

Epoch 1/500
21/21 [==============================] - 25s 527ms/step - loss: 1.6680 - accuracy: 0.3190 - val_loss: 1.2602 - val_accuracy: 0.2117 - lr: 0.0010
Epoch 2/500
21/21 [==============================] - 11s 505ms/step - loss: 1.2233 - accuracy: 0.4781 - val_loss: 0.9038 - val_accuracy: 0.6267 - lr: 0.0010
Epoch 3/500
21/21 [==============================] - 11s 507ms/step - loss: 0.9571 - accuracy: 0.5895 - val_loss: 0.8916 - val_accuracy: 0.2067 - lr: 0.0010
Epoch 4/500
21/21 [==============================] - 11s 506ms/step - loss: 0.8656 - accuracy: 0.6162 - val_loss: 0.6958 - val_accuracy: 0.7000 - lr: 0.0010
Epoch 5/500
21/21 [==============================] - 10s 495ms/step - loss: 0.7936 - accuracy: 0.6476 - val_loss: 0.6947 - val_accuracy: 0.6467 - lr: 0.0010
Epoch 6/500
21/21 [==============================] - 10s 494ms/step - loss: 0.7421 - accuracy: 0.6714 - val_loss: 0.7032 - val_accuracy: 0.6483 - lr: 0.0010
Epoch 7/500
21/21 [==============================] - 10s 5

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_2_0/assets


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_2_0/assets


[0.6966666666666667]
Epoch 1/500
21/21 [==============================] - 12s 526ms/step - loss: 1.7812 - accuracy: 0.3033 - val_loss: 1.7442 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/500
21/21 [==============================] - 10s 495ms/step - loss: 1.4957 - accuracy: 0.3514 - val_loss: 1.5769 - val_accuracy: 0.2150 - lr: 0.0010
Epoch 3/500
21/21 [==============================] - 11s 510ms/step - loss: 1.1646 - accuracy: 0.5348 - val_loss: 0.8119 - val_accuracy: 0.6600 - lr: 0.0010
Epoch 4/500
21/21 [==============================] - 10s 492ms/step - loss: 0.8544 - accuracy: 0.6295 - val_loss: 0.7770 - val_accuracy: 0.6783 - lr: 0.0010
Epoch 5/500
21/21 [==============================] - 10s 495ms/step - loss: 0.8054 - accuracy: 0.6571 - val_loss: 0.6938 - val_accuracy: 0.6350 - lr: 0.0010
Epoch 6/500
21/21 [==============================] - 11s 509ms/step - loss: 0.7315 - accuracy: 0.6871 - val_loss: 0.4678 - val_accuracy: 0.9650 - lr: 0.0010
Epoch 7/500
21/21 [==============

...

INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_2_8/assets


[0.6966666666666667, 0.7566666666666667, 0.74, 0.92, 0.9733333333333334, 0.9766666666666667, 0.8133333333333334, 0.9733333333333334, 0.79]
Epoch 1/500
21/21 [==============================] - 12s 526ms/step - loss: 1.8106 - accuracy: 0.3162 - val_loss: 1.7291 - val_accuracy: 0.2450 - lr: 0.0010
Epoch 2/500
21/21 [==============================] - 11s 508ms/step - loss: 1.4738 - accuracy: 0.3848 - val_loss: 1.6246 - val_accuracy: 0.2450 - lr: 0.0010
Epoch 3/500
21/21 [==============================] - 11s 510ms/step - loss: 1.2549 - accuracy: 0.4981 - val_loss: 1.0338 - val_accuracy: 0.2300 - lr: 0.0010
Epoch 4/500
21/21 [==============================] - 11s 508ms/step - loss: 1.0152 - accuracy: 0.6057 - val_loss: 0.9398 - val_accuracy: 0.2767 - lr: 0.0010
Epoch 5/500
21/21 [==============================] - 11s 510ms/step - loss: 0.8544 - accuracy: 0.6533 - val_loss: 0.7969 - val_accuracy: 0.2533 - lr: 0.0010
Epoch 6/500
21/21 [==============================] - 11s 508ms/step - loss: 

INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_2_9/assets


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_2_9/assets


[0.6966666666666667, 0.7566666666666667, 0.74, 0.92, 0.9733333333333334, 0.9766666666666667, 0.8133333333333334, 0.9733333333333334, 0.79, 0.9666666666666667]


In [18]:
metrics_accuracy = np.array(metrics_accuracy)
metrics_precision = np.array(metrics_precision)
metrics_recall = np.array(metrics_recall)
metrics_f1 = np.array(metrics_f1)

print(str('mean accuracy = ' + str(metrics_accuracy.mean() )))
print(str('mean precision = ' + str(metrics_precision.mean() )))
print(str('mean recall = ' + str(metrics_recall.mean() )))
print(str('mean f1 = ' + str(metrics_f1.mean() )))

print('***___***')

print(str('accuracy standard deviation = ' + str(metrics_accuracy.std() )))
print(str('precision standard deviation = ' + str(metrics_precision.std() )))
print(str('recall standard deviation = ' + str(metrics_recall.std() )))
print(str('f1 standard deviation = ' + str(metrics_f1.std() )))

print('***___***')

print(str('accuracy range = ' + str(metrics_accuracy.max() - metrics_accuracy.min() )))
print(str('precision range = ' + str(metrics_precision.max() - metrics_precision.min() )))
print(str('recall range = ' + str(metrics_recall.max() - metrics_recall.min() )))
print(str('f1 range = ' + str(metrics_f1.max() - metrics_f1.min() )))

print('***___***')

print(str('min accuracy = ' + str(metrics_accuracy.min() )))
print(str('min precision = ' + str(metrics_precision.min() )))
print(str('min recall = ' + str(metrics_recall.min() )))
print(str('min f1 = ' + str(metrics_f1.min() )))



mean accuracy = 0.8606666666666666
mean precision = 0.8044757841396477
mean recall = 0.8255841440518861
mean f1 = 0.796710261577
***___***
accuracy standard deviation = 0.10635161807263052
precision standard deviation = 0.17713148535199832
recall standard deviation = 0.1291956881280909
f1 standard deviation = 0.16192582339633305
***___***
accuracy range = 0.28
precision range = 0.40683902204825095
recall range = 0.3243343573988735
f1 range = 0.39353337839530333
***___***
min accuracy = 0.6966666666666667
min precision = 0.5686440697391887
min recall = 0.6451100870455709
min f1 = 0.5776193251103413


In [19]:
print(metrics_accuracy)
print(metrics_precision)
print(metrics_recall)
print(metrics_f1)

[0.69666667 0.75666667 0.74       0.92       0.97333333 0.97666667
 0.81333333 0.97333333 0.79       0.96666667]
[0.56864407 0.59405663 0.59518113 0.92584978 0.97355313 0.97548309
 0.86867759 0.97144883 0.60592257 0.96594101]
[0.64511009 0.69272913 0.6812212  0.89726916 0.96296296 0.96944444
 0.76261307 0.96428571 0.72279826 0.95740741]
[0.57761933 0.62484277 0.61408873 0.90205684 0.96635965 0.9711527
 0.73567547 0.9671926  0.64897113 0.95914339]
